In [61]:
import os
import time
import re
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
# 브랜드 설정 (금별 + 역전)
# =============================================
FILES = {
    "역전": r"F:\데이터분석\역전할머니_정제완료.csv",
    "금별": r"F:\데이터분석\금별맥주_정제완료.csv"
}

OUTPUT_DIR = r"F:\데이터분석\출력"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_FILES = {
    "역전": os.path.join(OUTPUT_DIR, "역전_방문자리뷰수.csv"),
    "금별": os.path.join(OUTPUT_DIR, "금별_방문자리뷰수.csv"),
}

In [63]:
# =============================================
# Selenium 드라이버 설정
# =============================================
def create_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(options=options)

    # webdriver 탐지 제거
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
            """
        },
    )
    return driver

driver = create_driver()
wait = WebDriverWait(driver, 15)

In [64]:
# 네이버 차단 여부 최소 체크
# =============================================
def is_blocked(html: str) -> bool:
    block_keywords = [
        "비정상적인 접근",
        "자동화된",
        "접근이 제한"
    ]
    return any(k in html for k in block_keywords)

In [65]:
# 방문자 리뷰 수 추출 함수
# =============================================
def get_visitor_review_count(place_id: int) -> int:
    url = f"https://pcmap.place.naver.com/place/{place_id}/review/visitor"
    driver.get(url)
    time.sleep(3)

    html = driver.page_source

    if is_blocked(html):
        raise Exception("네이버 차단 감지")

    soup = BeautifulSoup(html, "html.parser")

    # 방문자 리뷰 수: "245명 참여"
    span = soup.select_one("span.mholG")
    if not span:
        raise Exception("리뷰 태그 없음")

    count = re.sub(r"[^\d]", "", span.get_text())
    return int(count) if count else 0

In [66]:
# 메인 크롤링 실행 (체크포인트 5개)
# =============================================
CHECKPOINT_STEP = 5

try:
    for brand in ["역전", "금별"]:
        print("\n" + "=" * 40)
        print(f"[{brand}] 방문자 리뷰 수 크롤링 시작")
        print("=" * 40)

        df = pd.read_csv(FILES[brand])
        df["place_id"] = df["place_id"].astype(int)

        output_path = OUTPUT_FILES[brand]

        if os.path.exists(output_path):
            result_df = pd.read_csv(output_path)
            done_ids = set(result_df["place_id"])
        else:
            result_df = pd.DataFrame(columns=["place_id", "방문자_리뷰수"])
            done_ids = set()

        targets = [pid for pid in df["place_id"] if pid not in done_ids]
        total = len(targets)

        print(f"[{brand}] 대상 수: {total}")

        for idx, pid in enumerate(targets, 1):
            print(f"[{brand}] [{idx}/{total}] place_id={pid}", end=" → ")

            try:
                cnt = get_visitor_review_count(pid)
                print(cnt)
            except Exception as e:
                print(f"오류({e}) → 0 처리")
                cnt = 0

            result_df.loc[len(result_df)] = {
                "place_id": pid,
                "방문자_리뷰수": cnt
            }

            if idx % CHECKPOINT_STEP == 0:
                result_df.to_csv(output_path, index=False, encoding="utf-8-sig")
                print(f"[{brand}] 💾 체크포인트 저장")

            time.sleep(2)

        result_df.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"[{brand}] ✅ 완료")

finally:
    driver.quit()
    print("\n브라우저 종료")


[역전] 방문자 리뷰 수 크롤링 시작
[역전] 대상 수: 322
[역전] [1/322] place_id=1054587722 → 211
[역전] [2/322] place_id=1573115649 → 245
[역전] [3/322] place_id=1407805860 → 300
[역전] [4/322] place_id=1420076532 → 425
[역전] [5/322] place_id=1337289813 → 428
[역전] 💾 체크포인트 저장
[역전] [6/322] place_id=1059948798 → 198
[역전] [7/322] place_id=1199375408 → 321
[역전] [8/322] place_id=1679915153 → 274
[역전] [9/322] place_id=1125792665 → 855
[역전] [10/322] place_id=1424284773 → 161
[역전] 💾 체크포인트 저장
[역전] [11/322] place_id=1093149808 → 293
[역전] [12/322] place_id=1383332046 → 356
[역전] [13/322] place_id=1014475021 → 407
[역전] [14/322] place_id=1215644969 → 161
[역전] [15/322] place_id=1615941540 → 724
[역전] 💾 체크포인트 저장
[역전] [16/322] place_id=1071087271 → 409
[역전] [17/322] place_id=1617580186 → 409
[역전] [18/322] place_id=1105573661 → 875
[역전] [19/322] place_id=1893135961 → 25
[역전] [20/322] place_id=1412314791 → 533
[역전] 💾 체크포인트 저장
[역전] [21/322] place_id=1126054619 → 252
[역전] [22/322] place_id=1850542918 → 178
[역전] [23/322] place_id=100722

In [ ]:
# 리뷰수 0인거, 차단 감지된거 다시